# StyleGAN2 ADA PyTorch Generation

Made by [Artem Konevskikh](https://aiculedssul.net/)

Based on [dvschultz notebooks](https://github.com/dvschultz/ml-art-colabs)

Some pretrained models can be found [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2)

In [ ]:
#@title Install required libraries
!pip install ninja opensimplex numpy==1.21.6

In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to load pretrained models and to save the results.

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Stylegan2-ada
#@markdown StyleGAN2-ada will be installed to your Google Drive to speed up the training process

#@markdown Run this cell. If you’re already installed the repo, it will skip the installation process and change into the repo’s directory. If you haven’t installed it, it will install all the files necessary.
import os
import shlex
import numpy as np

if os.path.isdir("/content/drive/MyDrive/stylegan2ada"):
    %cd "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir stylegan2ada
    %cd stylegan2ada
    !git clone https://github.com/artem-konevskikh/stylegan2-ada-pytorch.git
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    # !mkdir pretrained
    # !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/artem-konevskikh/stylegan2-ada-pytorch.git
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../




In [ ]:
#@title Convert Legacy Model

#@markdown If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this workshop you do **not** need to use this cell.

#@markdown Path to model that you want to convert 
source_pkl = "" #@param {type: "string"}
#@markdown Path and file name to convert to.
dest_pkl = "" #@param {type: "string"}

source_pkl = shlex.quote(source_pkl)
dest_pkl = shlex.quote(dest_pkl)
!python legacy.py --source={source_pkl} --dest={dest_pkl}

In [ ]:
#@title Download Pretrained Model
#@markdown This will download the model trained on 1024x1024 faces from FFHQ Dataset
!wget -O /content/ffhq.pkl https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

# Generation

In [ ]:
#@title Generate Images
#@markdown Directory to save the generated images
outdir = "/content/drive/MyDrive/stylegan/results/2023-01-17" #@param {type: "string"}
#@markdown Path to the pretrained model. The most accurate way is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that here
network = "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/results/00001-maria2-11gb-gpu-noaug-resumecustom/network-snapshot-000004.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Generation Parameters**

#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Provide one number or comma-separated list of integers
seeds = "42,3333,80" #@param {type: "string"}
#@markdown Generate random images (`seeds` parameter will be ignored)
gen_random = True #@param {type: "boolean"}
#@markdown Amount of random images to generate
n_imgs =  200#@param {type: "integer"}
#@markdown ---
#@markdown **Generate Non-Square Images**

#@markdown It's possible to make the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Width
width = 1920 #@param {type: "integer"}
#@markdown Height
height = 1080 #@param {type: "integer"}
#@markdown Padding style to apply in the additional space
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = ','.join(str(s) for s in list(set(list(np.random.randint(4294967295, size=n_imgs)))))
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''


outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare}

In [ ]:
#@title Truncation Traversal

#@markdown Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#@markdown Directory to save the generated images
outdir = "/content/drive/MyDrive/workshops/digitalfutures/results/2" #@param {type: "string"}
#@markdown Path to the pretrained model. The most accurate way is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that here
network = "/content/drive/MyDrive/workshops/digitalfutures/wood-clt-200.pkl" #@param {type: "string"}

#@markdown Pass this only one seed. Pick a favorite from your generated images.
seed = 42  #@param {type: "integer"}
#@markdown Starting truncation value.
start = -1  #@param {type: "number"}
#@markdown Stopping truncation value. This should be larger than the start value. (Will probably break if its not).
stop = 1  #@param {type: "number"}
#@markdown How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)
increment = 0.01  #@param {type: "number"}

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --process="truncation" --outdir={outdir} --start={start} --stop={stop} --increment={increment} --seeds={seed} --network={network}

In [ ]:
#@title Interpolation
#@markdown Directory to save the generated images
outdir = "/content/drive/MyDrive/stylegan2ada/results/interpolation-test" #@param {type: "string"}
#@markdown Path to the pretrained model. The most accurate way is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that here
network = "/content/drive/MyDrive/stylegan2ada/wood-clr-40.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Generation Parameters**

#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 0.8 #@param {type: "number"}
#@markdown This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. Provide one number or comma-separated list of integers
seeds = "42,333" #@param {type: "string"}
#@markdown Generate random images (`seeds` parameter will be ignored)
gen_random = True #@param {type: "boolean"}
#@markdown Amount of random images to generate
n_imgs = 5 #@param {type: "integer"}
#@markdown Loop interpolation
gen_loop = True #@param {type: "boolean"}
#@markdown ---
#@markdown **Interpolation Parameters**

#@markdown Interpolation type
interpolation = 'linear' #@param ['linear', 'slerp']
#@markdown Latent space
space = 'z' #@param ['z', 'w']
#@markdown Interpolation steps. How many frames to produce
steps=60 #@param {type: "integer"}
#@markdown FPS, video framerate
fps=48 #@param {type: "integer"}
#@markdown ---
#@markdown **Generate Non-Square Images**

#@markdown It's possible to make the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.
gen_nonsquare = False #@param {type: "boolean"}
#@markdown Width
width = 1920 #@param {type: "integer"}
#@markdown Height
height = 1080 #@param {type: "integer"}
#@markdown Padding style to apply in the additional space
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


if gen_random:
  seeds = list(set(list(np.random.randint(4294967295, size=n_imgs))))
  if gen_loop:
    seeds.append(seeds[-1])
  seeds = ','.join(str(s) for s in seeds)
else:
  seeds = ','.join(str(s).strip() for s in seeds.split(','))
print("Seeds: ", seeds)

nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --seeds={seeds} --network={network} {nonsquare} --space={space} --process="interpolation" --interpolation={interpolation} --frames={steps} --fps={fps}

In [ ]:
#@title Interpolation loops
#@markdown Directory to save the generated images
outdir = "/content/drive/MyDrive/stylegan/results/2021-09-03" #@param {type: "string"}
#@markdown Path to the pretrained model. The most accurate way is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that here
network = "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/results/00001-maria2-11gb-gpu-noaug-resumecustom/network-snapshot-000004.pkl" #@param {type: "string"}
#@markdown ---
#@markdown **Generation Parameters**

#@markdown Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 
truncation = 1 #@param {type: "number"}

#@markdown ---
#@markdown **Interpolation Parameters**

#@markdown Interpolation type
interpolation = 'circularloop' #@param ['noiseloop', 'circularloop']
#@markdown Number of frames
frames = 1440 #@param {type: "integer"}
#@markdown This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `frames` can help determine how fast the video feels.
diameter = 900 #@param {type: "number"}
#@markdown FPS, video framerate
fps=24 #@param {type: "integer"}
#@markdown Starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point
random_seed = 42 #@param {type: "integer"}

#@markdown ---
#@markdown **Generate Non-Square Images**

#@markdown It's possible to make the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.
gen_nonsquare = True #@param {type: "boolean"}
#@markdown Width
width = 1920 #@param {type: "integer"}
#@markdown Height
height = 1080 #@param {type: "integer"}
#@markdown Padding style to apply in the additional space
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''

outdir = shlex.quote(outdir)
network = shlex.quote(network)
!python generate.py --outdir={outdir} --trunc={truncation} --network={network} {nonsquare} --process="interpolation" --interpolation={interpolation} --diameter={diameter} --random_seed={random_seed} --frames={frames} --fps={fps}

In [ ]:
#@title Flesh Digressions

#@markdown Producing Videos from circular loops of the constant and latent layers in StyleGAN2
#@markdown A .pkl of a StyleGAN network model
network = "/content/drive/MyDrive/stylegan2ada/stylegan2-ada-pytorch/results/00001-maria2-11gb-gpu-noaug-resumecustom/network-snapshot-000004.pkl" #@param {type: "string"}

#@markdown The truncation psi used in the generator
truncation = 1 #@param {type: "number"}

#@markdown The radius for the constant layer interpolation
radius_large=300.0 #@param {type: "number"}

#@markdown The radius for the latent space interpolation
radius_small=40.0 #@param {type: "number"}

#@markdown The value of the step/increment for the constant layer interpolation
step1=0.005 #@param {type: "number"}

#@markdown The value of the step/increment for the latent space interpolation
step2=0.0025 #@param {type: "number"}

#@markdown The length of the video in terms of circular interpolation, ex. default is 1.0, step2 is 0.0025, so video length is 400 frames. Recommended to keep at 1.0
video_length=1.0 #@param {type: "number"}

#@markdown It's possible to make the model to output images that are not square. This isn’t as good as training a rectangular model, but with the right model it can still look nice.
gen_nonsquare = True #@param {type: "boolean"}
#@markdown Width
width = 1920 #@param {type: "integer"}
#@markdown Height
height = 1080 #@param {type: "integer"}
#@markdown Padding style to apply in the additional space
scale_type = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']


nonsquare = f'--size={width}-{height} --scale-type={scale_type}' if gen_nonsquare else ''
network = shlex.quote(network)

!python flesh_digression.py --pkl $network --psi=$truncation --radius_small=$radius_small --radius_large=$radius_large --step1=$step1  --step2=$step2 $nonsquare